In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, lit, when, first, coalesce, 
    # Funções adicionadas para os novos cálculos
    floor, datediff, current_date, to_date, hour 
)
from pyspark.sql.types import DateType, StringType # Adicionado StringType

# Configurações do Spark (MANTIDAS)
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).appName("UnificarUPA").getOrCreate()

# ======================
# ARQUIVOS E CAMINHOS
# ======================

s3_trusted_path = 's3a://bucket-trusted-upa-connect-sofh/'
caminho_sensores = s3_trusted_path + 'tabela_sensores_tratada.csv'
caminho_atendimentos = s3_trusted_path + 'tabela_atendimentos_tratada.csv'
caminho_upa = s3_trusted_path + 'upa.csv'
caminho_paciente = s3_trusted_path + 'paciente.csv'

# ======================
# LEITURA DAS TABELAS
# ======================

# Tabela Sensores
sensores_df = spark.read.option('header', 'true').option('delimiter', ';').csv(caminho_sensores)

# Tabela Atendimentos
atendimento_df = spark.read.option('header', 'true').option('delimiter', ';').option('inferSchema', 'true').csv(caminho_atendimentos)

# Tabela UPA
upa_df = spark.read.option('header', 'true').option('delimiter', ',').csv(caminho_upa)

# Tabela Paciente
# =========================================================================
# !!! VERIFIQUE AQUI !!! 
# O delimitador do paciente.csv é ',' ou ';' (como os outros)?
# Se for ';', troque para .option('delimiter', ';')
# =========================================================================
paciente_df = spark.read.option('header', 'true').option('delimiter', ',').csv(caminho_paciente)


# ===============================================
# 1. TRATAMENTO/PIVOTAGEM DA TABELA SENSORES
# ===============================================

sensores_pivot_df = (
    sensores_df
    .withColumn('chave_pivot',
                when(col('fk_sensor') == 1, lit('camera_visao_computacional'))
                .when((col('fk_sensor') == 2) & (col('fk_unid_medida') == 1), lit('temperatura_ambiente'))
                .when((col('fk_sensor') == 2) & (col('fk_unid_medida') == 2), lit('umidade_ambiente'))
                .otherwise(lit('outro_sensor'))
               )
    .groupBy('data_hora', 'fk_upa')
    .pivot('chave_pivot')
    .agg(first('valor'))
)

campos_pivot = ['camera_visao_computacional', 'temperatura_ambiente', 'umidade_ambiente']
for campo in campos_pivot:
    if campo not in sensores_pivot_df.columns:
        sensores_pivot_df = sensores_pivot_df.withColumn(campo, lit(None))

sensores_final_df = sensores_pivot_df.drop('outro_sensor')

# ===============================================
# 2. JUNÇÕES (JOINs)
# ===============================================

# 2.1. Junção Sensores (FULL OUTER) Atendimentos
sensores_renomeado = sensores_final_df.withColumnRenamed('fk_upa', 'fk_upa_sensores')
atendimentos_renomeado = atendimento_df.withColumnRenamed('fk_upa', 'fk_upa_atendimentos')

df_juncao1 = sensores_renomeado.join(
    atendimentos_renomeado,
    on=sensores_renomeado.data_hora == atendimentos_renomeado.data_hora,
    how='full_outer'
).withColumn('data_hora_final', coalesce(sensores_renomeado.data_hora, atendimentos_renomeado.data_hora))

df_juncao1 = df_juncao1.withColumn('fk_upa_final', coalesce(col('fk_upa_sensores'), col('fk_upa_atendimentos')))

# Remove colunas duplicadas/intermediárias, exceto a FK consolidada
df_juncao1 = df_juncao1.drop(sensores_renomeado.data_hora).drop(atendimentos_renomeado.data_hora).drop('fk_upa_sensores').drop('fk_upa_atendimentos')
df_juncao1 = df_juncao1.withColumnRenamed('data_hora_final', 'data_hora')

# 2.2. Junção com UPA (LEFT JOIN)
upa_df_renomeado = upa_df.select(
    col('id_upa'),
    col('nome').alias('nome_upa'),
    col('capacidade_atendimento'),
    col('latitude'),
    col('longitude')
)

df_juncao2 = df_juncao1.join(
    upa_df_renomeado,
    on=df_juncao1.fk_upa_final == upa_df_renomeado.id_upa,
    how='left_outer'
)

# 2.3. Junção com Paciente (LEFT JOIN)
paciente_df_renomeado = paciente_df.select(
    col('id_paciente'),
    col('nome').alias('nome_paciente'),
    col('cpf'),
    col('data_nascimento'),
    col('biometria')
)

df_final = df_juncao2.join(
    paciente_df_renomeado,
    on=df_juncao2.fk_paciente == paciente_df_renomeado.id_paciente,
    how='left_outer'
).drop('fk_paciente')

# ===============================================
# 3. SELEÇÃO E RENOMEAÇÃO FINAL
# ===============================================

colunas_resultado = [
    col('data_hora'),
    col('id_upa'),
    col('nome_upa'),
    col('capacidade_atendimento'),
    col('latitude'),
    col('longitude'),
    col('camera_visao_computacional'),
    col('temperatura_ambiente'),
    col('umidade_ambiente'),
    col('temperatura_paciente'),
    col('oximetria_paciente'),
    col('id_paciente'),
    col('nome_paciente'),
    col('cpf'),
    col('data_nascimento'),
    col('biometria'),
    col('triagem_horario'),
    col('triagem_sala'),
    col('sala_de_espera').alias('sala_de_espera_horario'),
    col('consultorio_horario'),
    col('consultorio_sala'),
    col('saida').alias('saida_horario'),
]

df_final_selecionado = df_final.select(*colunas_resultado)

# ===============================================
# 4. ADIÇÃO DA COLUNA DE LEGENDA DE GRAVIDADE
# ===============================================


df_com_legenda = df_final_selecionado.withColumn('legenda_gravidade',
    # Adicionando a condição para verificar se oximetria_paciente OU temperatura_paciente é nulo
    when(col('oximetria_paciente').isNull() | col('temperatura_paciente').isNull(), F.lit(None).cast('string'))
    # As condições originais vêm em seguida
    .when(col('oximetria_paciente') < 87, 'Emergência - Oxigenação Crítica')
    .when(col('temperatura_paciente') > 39.5, 'Emergência - Febre Hipertermia')
    .when((col('temperatura_paciente') <= 35.0) & (col('oximetria_paciente') >= 87) & (col('oximetria_paciente') < 90), 'Alerta - Hipotermia e Hipoxemia')
    .when((col('temperatura_paciente') > 38.5) & (col('oximetria_paciente') >= 87) & (col('oximetria_paciente') <= 91), 'Alerta - Febre Alta e Oxigenação Reduzida')
    .when((col('temperatura_paciente') > 38.0) & (col('oximetria_paciente') >= 87) & (col('oximetria_paciente') <= 91), 'Atenção - Febre e Oximetria Reduzida')
    .when(col('temperatura_paciente') > 38.0, 'Atenção - Febre')
    .when((col('oximetria_paciente') >= 87) & (col('oximetria_paciente') <= 91), 'Atenção - Oxigenação Baixa')
    .when(col('temperatura_paciente') < 35.5, 'Atenção - Temperatura Baixa')
    .otherwise('Normal - Sem Alertas')
)

# ===============================================
# 5. CÁLCULO DE IDADE, FAIXA ETÁRIA E FAIXA DE HORA
# ===============================================

# 5.1. Calcular Idade e Faixa Etária
# Assegurar que 'data_nascimento' é tipo Data.
df_com_idade_hora = df_com_legenda.withColumn(
    "data_nascimento_dt",
    when(to_date(col("data_nascimento"), "yyyy-MM-dd").isNotNull(), to_date(col("data_nascimento"), "yyyy-MM-dd"))
    .when(to_date(col("data_nascimento"), "dd/MM/yyyy").isNotNull(), to_date(col("data_nascimento"), "dd/MM/yyyy"))
    .when(to_date(col("data_nascimento"), "MM/dd/yyyy").isNotNull(), to_date(col("data_nascimento"), "MM/dd/yyyy"))
    .otherwise(lit(None).cast(DateType())) # Caso nenhum formato corresponda
)

df_com_idade_hora = df_com_idade_hora.withColumn(
    "idade_paciente", floor(datediff(current_date(), col("data_nascimento_dt")) / 365.25)
)

# =========================================================================
# !!! CORREÇÃO DE LÓGICA AQUI !!!
# Adicionamos a verificação .isNull() primeiro.
# Se a idade for NULA, a faixa etária será NULA, e não "60+".
# =========================================================================
df_com_idade_hora = df_com_idade_hora.withColumn("faixa_etaria_paciente",
    when(col("idade_paciente").isNull(), lit(None).cast(StringType())) # <-- CORREÇÃO
    .when((col("idade_paciente") >= 0) & (col("idade_paciente") <= 12), "0–12") 
    .when((col("idade_paciente") >= 13) & (col("idade_paciente") <= 17), "13–17") 
    .when((col("idade_paciente") >= 18) & (col("idade_paciente") <= 39), "18–39") 
    .when((col("idade_paciente") >= 40) & (col("idade_paciente") <= 59), "40–59") 
    .otherwise("60+")) # Agora 'otherwise' só pega idades válidas >= 60

# 5.2. Criar Faixa de Horários (Usando a coluna 'data_hora' principal)
df_com_idade_hora = df_com_idade_hora.withColumn("Hora", hour(col("data_hora"))) # Usando 'data_hora'

df_com_idade_hora = df_com_idade_hora.withColumn("faixa_hora",
    when((col("Hora") >= 0) & (col("Hora") < 6), "0h-6h")
    .when((col("Hora") >= 6) & (col("Hora") < 9), "6h-9h")
    .when((col("Hora") >= 9) & (col("Hora") < 12), "9h-12h")
    .when((col("Hora") >= 12) & (col("Hora") < 15), "12h-15h")
    .when((col("Hora") >= 15) & (col("Hora") < 18), "15h-18h")
    .when((col("Hora") >= 18) & (col("Hora") < 21), "18h-21h")
    .otherwise("21h-24h"))

# 5.3. Remover colunas intermediárias
df_final_com_tudo = df_com_idade_hora.drop("data_nascimento_dt", "Hora")


# ===============================================
# 6. RESULTADO FINAL (MOSTRANDO O NOVO DF)
# ===============================================

print("Schema do DataFrame Final com Legenda, Idade e Faixas:")
df_final_com_tudo.printSchema()

print("\n5 primeiras linhas do DataFrame Final com Legenda, Idade e Faixas:")
df_final_com_tudo.show(5, truncate=False)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-441ba23d-24f7-44e4-a20d-dd3e4b2fd7da;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 343ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Schema do DataFrame Final com Legenda, Idade e Faixas:
root
 |-- data_hora: string (nullable = true)
 |-- id_upa: string (nullable = true)
 |-- nome_upa: string (nullable = true)
 |-- capacidade_atendimento: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- camera_visao_computacional: string (nullable = true)
 |-- temperatura_ambiente: string (nullable = true)
 |-- umidade_ambiente: string (nullable = true)
 |-- temperatura_paciente: double (nullable = true)
 |-- oximetria_paciente: integer (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- nome_paciente: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- biometria: string (nullable = true)
 |-- triagem_horario: timestamp (nullable = true)
 |-- triagem_sala: string (nullable = true)
 |-- sala_de_espera_horario: timestamp (nullable = true)
 |-- consultorio_horario: timestamp (nullable = true)
 |-- cons

25/10/25 22:11:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+------+--------------------------------------+----------------------+-------------------+-------------------+--------------------------+--------------------+----------------+--------------------+------------------+-----------+-------------+----+---------------+---------+---------------+------------+----------------------+-------------------+----------------+-------------+-----------------+--------------+---------------------+----------+
|data_hora          |id_upa|nome_upa                              |capacidade_atendimento|latitude           |longitude          |camera_visao_computacional|temperatura_ambiente|umidade_ambiente|temperatura_paciente|oximetria_paciente|id_paciente|nome_paciente|cpf |data_nascimento|biometria|triagem_horario|triagem_sala|sala_de_espera_horario|consultorio_horario|consultorio_sala|saida_horario|legenda_gravidade|idade_paciente|faixa_etaria_paciente|faixa_hora|
+-------------------+------+--------------------------------------+---------

In [2]:
df_final_com_tudo.coalesce(1) \
        .write \
        .option('header', 'true') \
        .mode('overwrite') \
        .csv('s3a://bucket-client-upa-connect-sofh/tabela_unificada_registros.csv')
print("Tabela salva com sucesso!")

25/10/25 22:11:14 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/25 22:11:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


Tabela salva com sucesso!
